In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

In [ ]:
# ▶️ seoul_abandoned_animal_status 등록 데이터 가져오기
query = """
SELECT district_level_2 AS district, dog_total, cat_total, dog_deceased, cat_deceased
FROM seoul_abandoned_animal_status
"""
df = pd.read_sql(query, engine)
df

,district,dog_total,cat_total,dog_deceased,cat_deceased
0,소계,2428,2307,565,1431
1,종로구,67,86,29,69
2,중구,33,51,5,35
3,용산구,116,109,3,61
4,성동구,53,107,8,58
5,광진구,79,84,12,57
6,동대문구,76,120,13,53
7,중랑구,104,105,12,80
8,성북구,88,84,21,69
9,강북구,140,81,48,52


In [ ]:
# ▶️ seoul_abandoned_animal_status 등록 데이터 가져오기
query = """
SELECT district_level_2 AS district, dog_total, cat_total, dog_deceased, cat_deceased
FROM seoul_abandoned_animal_status
"""
df = pd.read_sql(query, engine)
# 개 / 고양이 유기동물 발생 합계 계산
# 개 / 고양이 안락사 발생 합계 계산

# 0번행(총계) 제거
df.drop(0, inplace=True)

# 구별 개수 집계
df['abandon_count'] = df['dog_total'] + df['cat_total']
df['deceased_count'] = df['dog_deceased'] + df['cat_deceased']

# 2. 구별 유기동물/안락사 합계 집계
abandon_counts = df.groupby('district_level_2')['abandon_count'].sum().reset_index()
abandon_counts = abandon_counts.sort_values('abandon_count', ascending=False)

deceased_counts = df.groupby('district_level_2')['deceased_count'].sum().reset_index()
deceased_counts = deceased_counts.sort_values('deceased_count', ascending=False)

print(abandon_counts)
print(deceased_counts)

   district_level_2  abandon_count
12              마포구            323
4               관악구            261
1               강동구            258
21              은평구            227
20              용산구            225
3               강서구            222
2               강북구            221
24              중랑구            209
18              양천구            209
10             동대문구            196
17              송파구            195
8               노원구            182
6               구로구            178
9               도봉구            178
11              동작구            177
16              성북구            172
14              서초구            166
5               광진구            163
15              성동구            160
22              종로구            153
7               금천구            153
0               강남구            141
13             서대문구            141
19             영등포구            141
23               중구             84
   district_level_2  deceased_count
3               강서구             117
21              은평